In [2]:
import pandas as pd
import numpy as np

In [3]:
outfit = pd.read_csv("./outfit_2.csv")
outfit

,outfit_id,gender,style,season,tags,new_style,style_id
0,64453,F,스트릿,가을,"가을,반팔티셔츠,스트릿",F_스트릿,0
1,68758,F,캐주얼,봄,"스니커즈,캐주얼,나이키,봄",F_캐주얼,1
2,71478,F,캐주얼,여름,"여름,워커,셔츠,레더스커트,캐주얼",F_캐주얼,1
3,71480,F,로맨틱,여름,"로맨틱,여름,슬리브리스,스니커즈,숄더백,나이키,트랙팬츠",F_로맨틱,2
4,71482,M,스트릿,여름,"여름,스트릿,비니,데님팬츠,크로스백,스니커즈,나이키,베스트",M_스트릿,3
...,...,...,...,...,...,...,...
10424,92177,M,스트릿,여름,"여름,스트릿",M_스트릿,3
10425,92178,F,걸리시,여름,"여름,걸리시",F_걸리시,4
10426,92179,F,스트릿,여름,"여름,스트릿",F_스트릿,0
10427,92181,F,스트릿,여름,"여름,스트릿",F_스트릿,0


In [23]:
male = outfit[outfit['gender']=='M']['style_id'].values
female = outfit[outfit['gender']=='F']['style_id'].values

In [24]:
male

array([ 3,  3,  3, ...,  3, 13,  3], dtype=int64)

In [25]:
female

array([0, 1, 1, ..., 0, 0, 4], dtype=int64)

In [4]:
raw_like = pd.read_csv("./like_2.csv")
raw_like

,session_id,user_id,outfit_id,timestamp,style_id
0,66178898-dfb6-4de4-8f2a-cb09e3ccbe11,NaN,73016,23-07-19 08:53:25,5
1,66178898-dfb6-4de4-8f2a-cb09e3ccbe11,NaN,79507,23-07-19 08:53:25,10
2,7221dea7-71a9-4387-9a0e-96325662b63b,NaN,88572,23-07-19 08:53:53,5
3,7221dea7-71a9-4387-9a0e-96325662b63b,NaN,88731,23-07-19 08:53:54,12
4,7221dea7-71a9-4387-9a0e-96325662b63b,NaN,78346,23-07-19 08:53:55,3
...,...,...,...,...,...
1396,5faf94be-dc77-4133-aeff-20ec7c1b7511,NaN,80814,23-07-24 10:00:13,1
1397,84ac6b46-231c-4b30-8755-daa2e19f9661,NaN,80891,23-07-24 10:01:18,7
1398,84ac6b46-231c-4b30-8755-daa2e19f9661,NaN,89652,23-07-24 10:01:21,7
1399,84ac6b46-231c-4b30-8755-daa2e19f9661,NaN,81795,23-07-24 10:01:24,7


In [7]:
# 좋아요 2개 이상만 선택
session_cnt = raw_like['session_id'].value_counts()
valid_session = session_cnt[session_cnt > 1].index
like = raw_like[raw_like['session_id'].isin(valid_session)].copy()
session_map = {v:i for i,v in enumerate(like['session_id'].unique())}
like['session_id'] = like['session_id'].map(session_map)
like.drop(columns=['user_id'])
like

,session_id,user_id,outfit_id,timestamp,style_id
0,0,NaN,73016,23-07-19 08:53:25,5
1,0,NaN,79507,23-07-19 08:53:25,10
2,1,NaN,88572,23-07-19 08:53:53,5
3,1,NaN,88731,23-07-19 08:53:54,12
4,1,NaN,78346,23-07-19 08:53:55,3
...,...,...,...,...,...
1396,178,NaN,80814,23-07-24 10:00:13,1
1397,179,NaN,80891,23-07-24 10:01:18,7
1398,179,NaN,89652,23-07-24 10:01:21,7
1399,179,NaN,81795,23-07-24 10:01:24,7


In [8]:
# 좋아요 기록으로 후보 목록 생성
def transform(s: pd.Series):
    l = s.tolist()
    return [l[:i] if i != 0 else np.nan for i in range(len(l))]

like['cand'] = like.groupby('session_id')['style_id'].apply(transform).explode().values
like

,session_id,user_id,outfit_id,timestamp,style_id,cand
0,0,NaN,73016,23-07-19 08:53:25,5,NaN
1,0,NaN,79507,23-07-19 08:53:25,10,[5]
2,1,NaN,88572,23-07-19 08:53:53,5,NaN
3,1,NaN,88731,23-07-19 08:53:54,12,[5]
4,1,NaN,78346,23-07-19 08:53:55,3,"[5, 12]"
...,...,...,...,...,...,...
1396,178,NaN,80814,23-07-24 10:00:13,1,"[1, 0, 0, 0, 1, 0]"
1397,179,NaN,80891,23-07-24 10:01:18,7,NaN
1398,179,NaN,89652,23-07-24 10:01:21,7,[7]
1399,179,NaN,81795,23-07-24 10:01:24,7,"[7, 7]"


In [10]:
# 좋아요 기록으로 추천 목록 생성
def recommend(row):
    cand = row['cand']
    if cand is np.nan:
        return []
    rec = np.random.choice(cand, min(8, len(cand)), replace=True)
    return rec

like['rec'] = like.apply(recommend, axis=1)
like

,session_id,user_id,outfit_id,timestamp,style_id,cand,rec
0,0,NaN,73016,23-07-19 08:53:25,5,NaN,[]
1,0,NaN,79507,23-07-19 08:53:25,10,[5],[5]
2,1,NaN,88572,23-07-19 08:53:53,5,NaN,[]
3,1,NaN,88731,23-07-19 08:53:54,12,[5],[5]
4,1,NaN,78346,23-07-19 08:53:55,3,"[5, 12]","[12, 12]"
...,...,...,...,...,...,...,...
1396,178,NaN,80814,23-07-24 10:00:13,1,"[1, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 1]"
1397,179,NaN,80891,23-07-24 10:01:18,7,NaN,[]
1398,179,NaN,89652,23-07-24 10:01:21,7,[7],[7]
1399,179,NaN,81795,23-07-24 10:01:24,7,"[7, 7]","[7, 7]"


In [11]:
def precision(row):
    style_id = row['style_id']
    rec = row['rec']
    if style_id in rec:
        return 1
    else:
        return 0
    
like['prec'] = like.apply(precision, axis=1)
like

,session_id,user_id,outfit_id,timestamp,style_id,cand,rec,prec
0,0,NaN,73016,23-07-19 08:53:25,5,NaN,[],0
1,0,NaN,79507,23-07-19 08:53:25,10,[5],[5],0
2,1,NaN,88572,23-07-19 08:53:53,5,NaN,[],0
3,1,NaN,88731,23-07-19 08:53:54,12,[5],[5],0
4,1,NaN,78346,23-07-19 08:53:55,3,"[5, 12]","[12, 12]",0
...,...,...,...,...,...,...,...,...
1396,178,NaN,80814,23-07-24 10:00:13,1,"[1, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 1]",1
1397,179,NaN,80891,23-07-24 10:01:18,7,NaN,[],0
1398,179,NaN,89652,23-07-24 10:01:21,7,[7],[7],1
1399,179,NaN,81795,23-07-24 10:01:24,7,"[7, 7]","[7, 7]",1


In [12]:
like['prec'].value_counts(normalize=True)

prec
0    0.594161
1    0.405839
Name: proportion, dtype: float64